In [70]:
#Necessary Installs
!pip install factor_analyzer
!pip install pingouin pandas numpy

In [71]:
#Necessary Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer.rotator import Rotator
from factor_analyzer import FactorAnalyzer
import pingouin as pg
from numpy.linalg import inv
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from sklearn.impute import KNNImputer



In [72]:
raw_excel = "/content/data analysis_final data.xlsx"

## PISA Scores

In [73]:
# Read o excell
pisa = pd.read_excel(raw_excel, sheet_name = "PISA",
                     nrows=28)

# Change country name to simplify:
pisa['Countries'] = pisa['Countries'].replace({
    'Slovak Republic': 'Slovakia',
})
target_countries = pisa['Countries'].unique().tolist()

target_countries


['Austria',
 'Belgium',
 'Bulgaria',
 'Cyprus',
 'Czechia',
 'Germany',
 'Denmark',
 'Spain',
 'Estonia',
 'Finland',
 'France',
 'Greece',
 'Croatia',
 'Hungary',
 'Ireland',
 'Italy',
 'Lithuania',
 'Luxembourg',
 'Latvia',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Sweden']

In [74]:
# Add the years to a variable year
pisa_long = pisa.melt(
    id_vars=["Countries", "CNT"],
    value_vars=[2015, 2018, 2022],
    var_name="Year",
    value_name="PISA_score"
)
pisa_long

,Countries,CNT,Year,PISA_score
0,Austria,AUT,2015,493.907236
1,Belgium,BEL,2015,505.839337
2,Bulgaria,BGR,2015,441.968988
3,Cyprus,QCY,2015,NaN
4,Czechia,CZE,2015,501.502250
...,...,...,...,...
76,Portugal,PRT,2022,481.111545
77,Romania,ROU,2022,436.232140
78,Slovakia,SVK,2022,462.299851
79,Slovenia,SVN,2022,471.298496


In [75]:
# definir index para country_Year
pisa_long["index_id"] = (
    pisa_long["Countries"].str.replace(" ", "_")  # remove spaces if needed
    + "_" +
    pisa_long["Year"].astype(str)
)

# set index
pisa_long = pisa_long.set_index("index_id")

# remove redundent columns
pisa_long = pisa_long.drop(columns=["Countries", "CNT", "Year"])
pisa_long

,PISA_score
index_id,
Austria_2015,493.907236
Belgium_2015,505.839337
Bulgaria_2015,441.968988
Cyprus_2015,NaN
Czechia_2015,501.502250
...,...
Portugal_2022,481.111545
Romania_2022,436.232140
Slovakia_2022,462.299851


## Grad Tertiary Ed per 1000 Total

In [76]:
# Read o excell
grad_ternary_1000 = pd.read_excel(raw_excel, sheet_name='Grad Tertiary Ed per 1000 Total',
        skiprows = 9, nrows=41)

# Change the Time column to Country
grad_ternary_1000 = grad_ternary_1000.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
grad_ternary_1000 = grad_ternary_1000[~grad_ternary_1000["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
grad_ternary_1000 = grad_ternary_1000[['Country', '2015', '2018', '2022']]


# Filter the countries
grad_ternary_1000 = grad_ternary_1000[grad_ternary_1000['Country'].isin(target_countries)].copy()
grad_ternary_1000

,Country,2015,2018,2022
3,Belgium,79.6,82.3,86.9
4,Bulgaria,76,78.6,87
5,Czechia,74.3,63.3,63.6
6,Denmark,102.9,104.8,104.9
7,Germany,55.9,57,67.6
8,Estonia,58.4,56.7,65.7
9,Ireland,119.6,145.2,155.8
10,Greece,57.7,62.2,81.5
11,Spain,88.2,96.3,108
12,France,97.8,104.5,114.9


In [77]:
# Add the years to a variable Year and put country as index
grad_ternary_1000_long = grad_ternary_1000.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="grad_ternary_1000"
)

# define index to country_Year
grad_ternary_1000_long["index_id"] = grad_ternary_1000_long["Country"].str.replace(" ", "_") + "_" + grad_ternary_1000_long["Year"].astype(str)

# put index
grad_ternary_1000_long = grad_ternary_1000_long.set_index("index_id")

# drop redundant columns
grad_ternary_1000_long = grad_ternary_1000_long.drop(columns=['Country', 'Year'])

# change : to Nan
grad_ternary_1000_long = grad_ternary_1000_long.replace(':', np.nan)


grad_ternary_1000_long

/tmp/ipython-input-3077956263.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grad_ternary_1000_long = grad_ternary_1000_long.replace(':', np.nan)


,grad_ternary_1000
index_id,
Belgium_2015,79.6
Bulgaria_2015,76.0
Czechia_2015,74.3
Denmark_2015,102.9
Germany_2015,55.9
...,...
Romania_2022,65.7
Slovenia_2022,75.7
Slovakia_2022,61.8


## % Stud Tertiary Ed 20-24

In [78]:
# Read o excell
stu_ternary_100 = pd.read_excel(raw_excel, sheet_name='% Stud Tertiary Ed 20-24',
        skiprows = 9, nrows=41)

# Change the Time column to Country
stu_ternary_100 = stu_ternary_100.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
stu_ternary_100 = stu_ternary_100[~stu_ternary_100["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
stu_ternary_100 = stu_ternary_100[['Country', '2015', '2018', '2022']]


# Filter the countries
stu_ternary_100 = stu_ternary_100[stu_ternary_100['Country'].isin(target_countries)].copy()
stu_ternary_100

,Country,2015,2018,2022
3,Belgium,36.6,39.2,42
4,Bulgaria,38.9,39.2,42.8
5,Czechia,36.6,35.7,40.6
6,Denmark,38.1,37.6,38.1
7,Germany,29.3,30.2,32.7
8,Estonia,31.1,29.5,30.7
9,Ireland,35.1,36.7,37.4
10,Greece,40.7,46.2,52.1
11,Spain,38.4,40.3,40.5
12,France,32.5,35.2,36.8


In [79]:
# Add the years to a variable Year and put country as index
stu_ternary_100_long = stu_ternary_100.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="stu_ternary_100"
)

# define index to country_Year
stu_ternary_100_long["index_id"] = stu_ternary_100_long["Country"].str.replace(" ", "_") + "_" + stu_ternary_100_long["Year"].astype(str)

# put index
stu_ternary_100_long = stu_ternary_100_long.set_index("index_id")

# drop redundant columns
stu_ternary_100_long = stu_ternary_100_long.drop(columns=['Country', 'Year'])

# change : to Nan
stu_ternary_100_long = stu_ternary_100_long.replace(':', np.nan)


stu_ternary_100_long

/tmp/ipython-input-2220237961.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stu_ternary_100_long = stu_ternary_100_long.replace(':', np.nan)


,stu_ternary_100
index_id,
Belgium_2015,36.6
Bulgaria_2015,38.9
Czechia_2015,36.6
Denmark_2015,38.1
Germany_2015,29.3
...,...
Romania_2022,31.9
Slovenia_2022,47.4
Slovakia_2022,30.8


## % Pop 18+ in Ed or Training

In [80]:
# Read o excell
adults_edu_train = pd.read_excel(raw_excel, sheet_name='% Pop 18+ in Ed or Training',
        skiprows = 9, nrows=40)

# Change the Time column to Country
adults_edu_train = adults_edu_train.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
adults_edu_train = adults_edu_train[~adults_edu_train["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
adults_edu_train = adults_edu_train[['Country', '2015', '2018', '2022']]


# Filter the countries
adults_edu_train = adults_edu_train[adults_edu_train['Country'].isin(target_countries)].copy()
adults_edu_train

,Country,2015,2018,2022
3,Belgium,13.8,15.9,18.2
4,Bulgaria,7.9,8.2,7.6
5,Czechia,14.5,14.1,14.9
6,Denmark,37.4,29.5,33.1
7,Germany,14.8,14.9,14.4
8,Estonia,17.5,23.6,25.6
9,Ireland,12,18.4,18.6
10,Greece,9.7,10.9,9.9
11,Spain,15.6,16.2,21.2
12,France,24.3,24.2,19.8


In [81]:
# Add the years to a variable Year and put country as index
adults_edu_train_long = adults_edu_train.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="a_edu_train_per_100"
)

# define index to country_Year
adults_edu_train_long["index_id"] = adults_edu_train_long["Country"].str.replace(" ", "_") + "_" + adults_edu_train_long["Year"].astype(str)

# put index
adults_edu_train_long = adults_edu_train_long.set_index("index_id")

# drop redundant columns
adults_edu_train_long = adults_edu_train_long.drop(columns=['Country', 'Year'])

# change : to Nan
adults_edu_train_long = adults_edu_train_long.replace(':', np.nan)


adults_edu_train_long

,a_edu_train_per_100
index_id,
Belgium_2015,13.8
Bulgaria_2015,7.9
Czechia_2015,14.5
Denmark_2015,37.4
Germany_2015,14.8
...,...
Romania_2022,11.2
Slovenia_2022,28.5
Slovakia_2022,18.5


## % Employees in Ed or Training

In [82]:
# Read o excell
emp_edu_train_100 = pd.read_excel(raw_excel, sheet_name='% Employees in Ed or Training',
        skiprows = 10, nrows=40)

# Change the Time column to Country
emp_edu_train_100 = emp_edu_train_100.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
emp_edu_train_100 = emp_edu_train_100[~emp_edu_train_100["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
emp_edu_train_100 = emp_edu_train_100[['Country', '2015', '2018', '2022']]


# Filter the countries
emp_edu_train_100 = emp_edu_train_100[emp_edu_train_100['Country'].isin(target_countries)].copy()
emp_edu_train_100


,Country,2015,2018,2022
3,Belgium,6.9,8.6,10.6
4,Bulgaria,2.3,2.6,1.8
5,Czechia,10.3,9.8,10.8
6,Denmark,32.7,23.2,28.5
7,Germany,11.9,11.5,11
8,Estonia,13.5,22.2,24.4
9,Ireland,5.9,13.2,13.4
10,Greece,4.3,5.7,3.2
11,Spain,11,11.2,17.1
12,France,22.8,22.7,17.3


In [83]:
# Add the years to a variable Year and put country as index
emp_edu_train_100_long = emp_edu_train_100.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="e_edu_train_per_100"
)

# define index to country_Year
emp_edu_train_100_long["index_id"] = emp_edu_train_100_long["Country"].str.replace(" ", "_") + "_" + emp_edu_train_100_long["Year"].astype(str)

# put index
emp_edu_train_100_long = emp_edu_train_100_long.set_index("index_id")

# drop redundant columns
emp_edu_train_100_long = emp_edu_train_100_long.drop(columns=['Country', 'Year'])

# change : to Nan
emp_edu_train_100_long = emp_edu_train_100_long.replace(':', np.nan)


emp_edu_train_100_long

,e_edu_train_per_100
index_id,
Belgium_2015,6.9
Bulgaria_2015,2.3
Czechia_2015,10.3
Denmark_2015,32.7
Germany_2015,11.9
...,...
Romania_2022,8.1
Slovenia_2022,26.2
Slovakia_2022,16.3


## Income by Ed Attainment (0-2)

In [84]:
# Read o excell
income_0_2 = pd.read_excel(raw_excel, sheet_name='Income by Ed Attainment (0-2)',
        skiprows = 11, nrows=47)

# Change the Time column to Country
income_0_2 = income_0_2.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
income_0_2 = income_0_2[~income_0_2["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
income_0_2 = income_0_2[['Country', '2015', '2018', '2022']]


# Filter the countries
income_0_2 = income_0_2[income_0_2['Country'].isin(target_countries)].copy()
income_0_2

,Country,2015,2018,2022
9,Belgium,16767,17672.0,20419
10,Bulgaria,1975,2170.0,3046
11,Czechia,6181,7590.0,10889
12,Denmark,26886,28299.0,30407
13,Germany,15707,17285.0,20200
14,Estonia,6356,8646.0,12146
15,Ireland,17181,19529.0,23856
16,Greece,5812,6158.0,7336
17,Spain,10431,11601.0,13556
18,France,17764,19291.0,18170


In [85]:
# Add the years to a variable Year and put country as index
income_0_2_long = income_0_2.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="income_0_2"
)

# define index to country_Year
income_0_2_long["index_id"] = income_0_2_long["Country"].str.replace(" ", "_") + "_" + income_0_2_long["Year"].astype(str)

# put index
income_0_2_long = income_0_2_long.set_index("index_id")

# drop redundant columns
income_0_2_long = income_0_2_long.drop(columns=['Country', 'Year'])

# change : to Nan
income_0_2_long = income_0_2_long.replace(':', np.nan)


income_0_2_long

,income_0_2
index_id,
Belgium_2015,16767
Bulgaria_2015,1975
Czechia_2015,6181
Denmark_2015,26886
Germany_2015,15707
...,...
Romania_2022,3269
Slovenia_2022,13648
Slovakia_2022,5615


## Income by Ed Attainment (3-4)

In [86]:
# Read o excell
income_3_4 = pd.read_excel(raw_excel, sheet_name='Income by Ed Attainment (3-4)',
        skiprows = 11, nrows=47)

# Change the Time column to Country
income_3_4 = income_3_4.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
income_3_4 = income_3_4[~income_3_4["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
income_3_4 = income_3_4[['Country', '2015', '2018', '2022']]


# Filter the countries
income_3_4 = income_3_4[income_3_4['Country'].isin(target_countries)].copy()
income_3_4

,Country,2015,2018,2022
9,Belgium,21920,24132.0,27148
10,Bulgaria,3763,4000.0,6004
11,Czechia,7759,9628.0,12922
12,Denmark,29255,31092.0,34546
13,Germany,20851,22105.0,25874
14,Estonia,8119,10572.0,15057
15,Ireland,20285,24367.0,29891
16,Greece,7300,7660.0,9325
17,Spain,13748,14882.0,16320
18,France,20455,21513.0,22453


In [87]:
# Add the years to a variable Year and put country as index
income_3_4_long = income_3_4.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="income_3_4"
)

# define index to country_Year
income_3_4_long["index_id"] = income_3_4_long["Country"].str.replace(" ", "_") + "_" + income_3_4_long["Year"].astype(str)

# put index
income_3_4_long = income_3_4_long.set_index("index_id")

# drop redundant columns
income_3_4_long = income_3_4_long.drop(columns=['Country', 'Year'])

# change : to Nan
income_3_4_long = income_3_4_long.replace(':', np.nan)


income_3_4_long

,income_3_4
index_id,
Belgium_2015,21920
Bulgaria_2015,3763
Czechia_2015,7759
Denmark_2015,29255
Germany_2015,20851
...,...
Romania_2022,5670
Slovenia_2022,16287
Slovakia_2022,9171


## Income by Ed Attainment (5-8)

In [88]:
# Read o excell
income_5_8 = pd.read_excel(raw_excel, sheet_name='Income by Ed Attainment (5-8)',
        skiprows = 11, nrows=47)

# Change the Time column to Country
income_5_8 = income_5_8.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
income_5_8 = income_5_8[~income_5_8["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
income_5_8 = income_5_8[['Country', '2015', '2018', '2022']]


# Filter the countries
income_5_8 = income_5_8[income_5_8['Country'].isin(target_countries)].copy()
income_5_8

,Country,2015,2018,2022
9,Belgium,28653,30644.0,34003
10,Bulgaria,5568,6015.0,9248
11,Czechia,10054,12331.0,16560
12,Denmark,34470,35774.0,38619
13,Germany,27321,28728.0,34335
14,Estonia,11364,14717.0,19754
15,Ireland,28632,31068.0,36444
16,Greece,10613,10850.0,12493
17,Spain,19187,20893.0,22300
18,France,26720,27036.0,30231


In [89]:
# Add the years to a variable Year and put country as index
income_5_8_long = income_5_8.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="income_5_8"
)

# define index to country_Year
income_5_8_long["index_id"] = income_5_8_long["Country"].str.replace(" ", "_") + "_" + income_5_8_long["Year"].astype(str)

# put index
income_5_8_long = income_5_8_long.set_index("index_id")

# drop redundant columns
income_5_8_long = income_5_8_long.drop(columns=['Country', 'Year'])

# change : to Nan
income_5_8_long = income_5_8_long.replace(':', np.nan)


income_5_8_long

,income_5_8
index_id,
Belgium_2015,28653
Bulgaria_2015,5568
Czechia_2015,10054
Denmark_2015,34470
Germany_2015,27321
...,...
Romania_2022,9591
Slovenia_2022,20326
Slovakia_2022,10431


## Ed. Gov.expenditure %GovExpend

In [90]:
# Read o excell
gov_exp = pd.read_excel(raw_excel, sheet_name='Ed. Gov.expenditure %GovExpend',
        skiprows = 0, nrows=29)

# Change column to Country
gov_exp = gov_exp.rename(columns={"Country Name": "Country"})

# Keep only the relevant columns
gov_exp = gov_exp[['Country', '2015', '2018', '2022']]


# Filter the countries
gov_exp = gov_exp[gov_exp['Country'].isin(target_countries)].copy()
gov_exp

,Country,2015,2018,2022
0,Austria,10.72222,10.68115,9.93358
1,Belgium,12.00727,12.18058,12.01736
2,Bulgaria,NaN,11.79003,11.98185
3,Croatia,NaN,8.73955,NaN
4,Cyprus,14.65875,NaN,12.48295
5,Czechia,13.71592,10.50821,10.00936
6,Denmark,NaN,14.43713,14.15744
7,Estonia,12.86756,13.23888,13.02919
9,Finland,12.69234,11.94185,12.11926
10,France,NaN,9.62476,9.11748


In [91]:
# Add the years to a variable Year and put country as index
gov_exp_long = gov_exp.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="gov_exp"
)

# define index to country_Year
gov_exp_long["index_id"] = gov_exp_long["Country"].str.replace(" ", "_") + "_" + gov_exp_long["Year"].astype(str)

# put index
gov_exp_long = gov_exp_long.set_index("index_id")

# drop redundant columns
gov_exp_long = gov_exp_long.drop(columns=['Country', 'Year'])

# change : to Nan
gov_exp_long = gov_exp_long.replace(':', np.nan)


gov_exp_long

,gov_exp
index_id,
Austria_2015,10.72222
Belgium_2015,12.00727
Bulgaria_2015,NaN
Croatia_2015,NaN
Cyprus_2015,14.65875
...,...
Portugal_2022,10.38977
Romania_2022,8.77512
Slovenia_2022,11.08095


## Output per hour worked

In [92]:
# Read o excell
prod = pd.read_excel(raw_excel, sheet_name='Output per hour worked',
        skiprows = 3, nrows=83)

# Change column to Country
prod = prod.rename(columns={"Row Labels": "Country"})

# Keep only the relevant columns
prod = prod[['Country', '2015', '2018', '2022']]


# Filter the countries
prod = prod[prod['Country'].isin(target_countries)].copy()
prod

,Country,2015,2018,2022
0,Austria,80.06,82.30,86.61
3,Belgium,85.44,84.81,89.71
6,Bulgaria,27.93,29.50,33.44
9,Croatia,42.56,46.46,52.77
12,Cyprus,32.25,33.21,36.49
15,Czechia,48.49,52.10,55.04
18,Denmark,84.81,88.36,91.57
21,Estonia,40.71,45.66,52.01
24,Finland,76.50,78.29,80.77
27,France,78.25,81.28,79.96


In [93]:
# Add the years to a variable Year and put country as index
prod_long = prod.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="prod"
)

# define index to country_Year
prod_long["index_id"] = prod_long["Country"].str.replace(" ", "_") + "_" + prod_long["Year"].astype(str)

# put index
prod_long = prod_long.set_index("index_id")

# drop redundant columns
prod_long = prod_long.drop(columns=['Country', 'Year'])

# change : to Nan
prod_long = prod_long.replace(':', np.nan)


prod_long

,prod
index_id,
Austria_2015,80.06
Belgium_2015,85.44
Bulgaria_2015,27.93
Croatia_2015,42.56
Cyprus_2015,32.25
...,...
Romania_2022,46.74
Slovakia_2022,45.33
Slovenia_2022,56.69


## Foreign Ed. Attainment 0-2

In [94]:
# Read o excell
foreign_ed_0_2 = pd.read_excel(raw_excel, sheet_name='Foreign Ed. Attainment 0-2',
        skiprows = 11, nrows=41)

# Change the Time column to Country
foreign_ed_0_2 = foreign_ed_0_2.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
foreign_ed_0_2 = foreign_ed_0_2[~foreign_ed_0_2["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
foreign_ed_0_2 = foreign_ed_0_2[['Country', '2015', '2018', '2022']]


# Filter the countries
foreign_ed_0_2 = foreign_ed_0_2[foreign_ed_0_2['Country'].isin(target_countries)].copy()
foreign_ed_0_2

,Country,2015,2018,2022
3,Belgium,37,34.1,29
4,Bulgaria,:,:,:
5,Czechia,14.6,15,11.6
6,Denmark,26.4,27.1,28.4
7,Germany,33.6,33.2,37.9
8,Estonia,6.7,8.1,6
9,Ireland,10.1,9.3,6.8
10,Greece,42.9,37.6,31.7
11,Spain,42.7,41.2,42.5
12,France,38.6,36.7,31.7


In [95]:
# Add the years to a variable Year and put country as index
foreign_ed_0_2_long = foreign_ed_0_2.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="foreign_ed_0_2"
)

# define index to country_Year
foreign_ed_0_2_long["index_id"] = foreign_ed_0_2_long["Country"].str.replace(" ", "_") + "_" + foreign_ed_0_2_long["Year"].astype(str)

# put index
foreign_ed_0_2_long = foreign_ed_0_2_long.set_index("index_id")

# drop redundant columns
foreign_ed_0_2_long = foreign_ed_0_2_long.drop(columns=['Country', 'Year'])

# change : to Nan
foreign_ed_0_2_long = foreign_ed_0_2_long.replace(':', np.nan)


foreign_ed_0_2_long

/tmp/ipython-input-3515912474.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  foreign_ed_0_2_long = foreign_ed_0_2_long.replace(':', np.nan)


,foreign_ed_0_2
index_id,
Belgium_2015,37.0
Bulgaria_2015,NaN
Czechia_2015,14.6
Denmark_2015,26.4
Germany_2015,33.6
...,...
Romania_2022,NaN
Slovenia_2022,21.1
Slovakia_2022,NaN


## Foreign Ed. Attainment 3-4

In [96]:
# Read o excell
foreign_ed_3_4 = pd.read_excel(raw_excel, sheet_name='Foreign Ed. Attainment 3-4',
        skiprows = 11, nrows=41)

# Change the Time column to Country
foreign_ed_3_4 = foreign_ed_3_4.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
foreign_ed_3_4 = foreign_ed_3_4[~foreign_ed_3_4["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
foreign_ed_3_4 = foreign_ed_3_4[['Country', '2015', '2018', '2022']]


# Filter the countries
foreign_ed_3_4 = foreign_ed_3_4[foreign_ed_3_4['Country'].isin(target_countries)].copy()
foreign_ed_3_4

,Country,2015,2018,2022
3,Belgium,32.8,32.2,32.9
4,Bulgaria,53.1,47.2,52.6
5,Czechia,59.4,54,54.8
6,Denmark,36.6,33.9,33.1
7,Germany,45,43.2,36.4
8,Estonia,52.8,49.1,43.3
9,Ireland,38.8,38,33.9
10,Greece,42.1,46.7,48.5
11,Spain,32.5,33.1,31.3
12,France,32.1,32.3,33


In [97]:
# Add the years to a variable Year and put country as index
foreign_ed_3_4_long = foreign_ed_3_4.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="foreign_ed_3_4"
)

# define index to country_Year
foreign_ed_3_4_long["index_id"] = foreign_ed_3_4_long["Country"].str.replace(" ", "_") + "_" + foreign_ed_3_4_long["Year"].astype(str)

# put index
foreign_ed_3_4_long = foreign_ed_3_4_long.set_index("index_id")

# drop redundant columns
foreign_ed_3_4_long = foreign_ed_3_4_long.drop(columns=['Country', 'Year'])

# change : to Nan
foreign_ed_3_4_long = foreign_ed_3_4_long.replace(':', np.nan)


foreign_ed_3_4_long

/tmp/ipython-input-356008522.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  foreign_ed_3_4_long = foreign_ed_3_4_long.replace(':', np.nan)


,foreign_ed_3_4
index_id,
Belgium_2015,32.8
Bulgaria_2015,53.1
Czechia_2015,59.4
Denmark_2015,36.6
Germany_2015,45.0
...,...
Romania_2022,44.3
Slovenia_2022,59.2
Slovakia_2022,57.2


## Foreign Ed. Attainment 5-8

In [98]:
# Read o excell
foreign_ed_5_8 = pd.read_excel(raw_excel, sheet_name='Foreign Ed. Attainment 5-8',
        skiprows = 11, nrows=41)

# Change the Time column to Country
foreign_ed_5_8 = foreign_ed_5_8.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
foreign_ed_5_8 = foreign_ed_5_8[~foreign_ed_5_8["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
foreign_ed_5_8 = foreign_ed_5_8[['Country', '2015', '2018', '2022']]


# Filter the countries
foreign_ed_5_8 = foreign_ed_5_8[foreign_ed_5_8['Country'].isin(target_countries)].copy()
foreign_ed_5_8

,Country,2015,2018,2022
3,Belgium,30.2,33.7,38.1
4,Bulgaria,40.6,48.3,44.3
5,Czechia,26,31,33.6
6,Denmark,37,38.9,38.6
7,Germany,21.4,23.7,25.7
8,Estonia,40.5,42.8,50.7
9,Ireland,51.1,52.7,59.4
10,Greece,15,15.7,19.8
11,Spain,24.8,25.7,26.2
12,France,29.3,30.9,35.4


In [99]:
# Add the years to a variable Year and put country as index
foreign_ed_5_8_long = foreign_ed_5_8.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="foreign_ed_5_8"
)

# define index to country_Year
foreign_ed_5_8_long["index_id"] = foreign_ed_5_8_long["Country"].str.replace(" ", "_") + "_" + foreign_ed_5_8_long["Year"].astype(str)

# put index
foreign_ed_5_8_long = foreign_ed_5_8_long.set_index("index_id")

# drop redundant columns
foreign_ed_5_8_long = foreign_ed_5_8_long.drop(columns=['Country', 'Year'])

# change : to Nan
foreign_ed_5_8_long = foreign_ed_5_8_long.replace(':', np.nan)


foreign_ed_5_8_long

/tmp/ipython-input-673056237.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  foreign_ed_5_8_long = foreign_ed_5_8_long.replace(':', np.nan)


,foreign_ed_5_8
index_id,
Belgium_2015,30.2
Bulgaria_2015,40.6
Czechia_2015,26.0
Denmark_2015,37.0
Germany_2015,21.4
...,...
Romania_2022,45.6
Slovenia_2022,19.6
Slovakia_2022,33.0


## Young People NEET

In [100]:
# Read o excell
NEET = pd.read_excel(raw_excel, sheet_name='Young People NEET',
        skiprows = 9, nrows=40)

# Change the Time column to Country
NEET = NEET.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
NEET = NEET[~NEET["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
NEET = NEET[['Country', '2015', '2018', '2022']]


# Filter the countries
NEET = NEET[NEET['Country'].isin(target_countries)].copy()
NEET

,Country,2015,2018,2022
3,Belgium,13.6,11.4,9.2
4,Bulgaria,22.2,18.1,14.8
5,Czechia,11.8,9.5,11.4
6,Denmark,8.5,9.6,7.9
7,Germany,9.3,8.7,8.6
8,Estonia,12.4,11.5,10.6
9,Ireland,16.5,11.6,8.6
10,Greece,23.8,19.2,15.3
11,Spain,19.4,15.3,12.7
12,France,13.9,12.9,12


In [101]:
# Add the years to a variable Year and put country as index
NEET_long = NEET.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="NEET"
)

# define index to country_Year
NEET_long["index_id"] = NEET_long["Country"].str.replace(" ", "_") + "_" + NEET_long["Year"].astype(str)

# put index
NEET_long = NEET_long.set_index("index_id")

# drop redundant columns
NEET_long = NEET_long.drop(columns=['Country', 'Year'])

# change : to Nan
NEET_long = NEET_long.replace(':', np.nan)


NEET_long

,NEET
index_id,
Belgium_2015,13.6
Bulgaria_2015,22.2
Czechia_2015,11.8
Denmark_2015,8.5
Germany_2015,9.3
...,...
Romania_2022,19.8
Slovenia_2022,8.4
Slovakia_2022,12.3


## Employment Ed Attainment 0-2

In [102]:
# Read o excell
emp_ed_0_2 = pd.read_excel(raw_excel, sheet_name='Employment Ed Attainment 0-2',
        skiprows = 10, nrows=40)

# Change the Time column to Country
emp_ed_0_2 = emp_ed_0_2.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
emp_ed_0_2 = emp_ed_0_2[~emp_ed_0_2["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
emp_ed_0_2 = emp_ed_0_2[['Country', '2015', '2018', '2022']]


# Filter the countries
emp_ed_0_2 = emp_ed_0_2[emp_ed_0_2['Country'].isin(target_countries)].copy()
emp_ed_0_2

,Country,2015,2018,2022
3,Belgium,15.3,12.7,10.9
4,Bulgaria,10.1,10.7,10.2
5,Czechia,3.8,4.1,4
6,Denmark,15.7,14.7,14.6
7,Germany,11.8,12.1,13.8
8,Estonia,9.9,10.3,8.8
9,Ireland,12,10.6,7.7
10,Greece,26.7,22.6,15.2
11,Spain,34,32.8,29.8
12,France,15.8,14.4,11.5


In [103]:
# Add the years to a variable Year and put country as index
emp_ed_0_2_long = emp_ed_0_2.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="emp_ed_0_2"
)

# define index to country_Year
emp_ed_0_2_long["index_id"] = emp_ed_0_2_long["Country"].str.replace(" ", "_") + "_" + emp_ed_0_2_long["Year"].astype(str)

# put index
emp_ed_0_2_long = emp_ed_0_2_long.set_index("index_id")

# drop redundant columns
emp_ed_0_2_long = emp_ed_0_2_long.drop(columns=['Country', 'Year'])

# change : to Nan
emp_ed_0_2_long = emp_ed_0_2_long.replace(':', np.nan)


emp_ed_0_2_long

,emp_ed_0_2
index_id,
Belgium_2015,15.3
Bulgaria_2015,10.1
Czechia_2015,3.8
Denmark_2015,15.7
Germany_2015,11.8
...,...
Romania_2022,11.8
Slovenia_2022,5.5
Slovakia_2022,2.7


## Employment Ed Attainment 3-4

In [104]:
# Read o excell
emp_ed_3_4 = pd.read_excel(raw_excel, sheet_name='Employment Ed Attainment 3-4',
        skiprows = 10, nrows=40)

# Change the Time column to Country
emp_ed_3_4 = emp_ed_3_4.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
emp_ed_3_4 = emp_ed_3_4[~emp_ed_3_4["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
emp_ed_3_4 = emp_ed_3_4[['Country', '2015', '2018', '2022']]


# Filter the countries
emp_ed_3_4 = emp_ed_3_4[emp_ed_3_4['Country'].isin(target_countries)].copy()
emp_ed_3_4

,Country,2015,2018,2022
3,Belgium,38.3,37.5,37
4,Bulgaria,56.7,56.7,56.3
5,Czechia,72.8,70.9,69
6,Denmark,46.1,44.5,42.9
7,Germany,57.1,55.8,53
8,Estonia,51.6,49.9,48.4
9,Ireland,35.8,36.7,34.5
10,Greece,39.7,40.5,45.2
11,Spain,23.9,23.9,23.8
12,France,44.6,43.1,41.4


In [105]:
# Add the years to a variable Year and put country as index
emp_ed_3_4_long = emp_ed_3_4.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="emp_ed_3_4"
)

# define index to country_Year
emp_ed_3_4_long["index_id"] = emp_ed_3_4_long["Country"].str.replace(" ", "_") + "_" + emp_ed_3_4_long["Year"].astype(str)

# put index
emp_ed_3_4_long = emp_ed_3_4_long.set_index("index_id")

# drop redundant columns
emp_ed_3_4_long = emp_ed_3_4_long.drop(columns=['Country', 'Year'])

# change : to Nan
emp_ed_3_4_long = emp_ed_3_4_long.replace(':', np.nan)


emp_ed_3_4_long

,emp_ed_3_4
index_id,
Belgium_2015,38.3
Bulgaria_2015,56.7
Czechia_2015,72.8
Denmark_2015,46.1
Germany_2015,57.1
...,...
Romania_2022,63.8
Slovenia_2022,51
Slovakia_2022,65.6


## Employment Ed Attainment 5-8

In [106]:
# Read o excell
emp_ed_5_8 = pd.read_excel(raw_excel, sheet_name='Employment Ed Attainment 5-8',
        skiprows = 10, nrows=40)

# Change the Time column to Country
emp_ed_5_8 = emp_ed_5_8.rename(columns={"TIME": "Country"})

# removing the old row from Geo because is all null
emp_ed_5_8 = emp_ed_5_8[~emp_ed_5_8["Country"].isin(['GEO (Labels)'])]

# Keep only the relevant columns
emp_ed_5_8 = emp_ed_5_8[['Country', '2015', '2018', '2022']]


# Filter the countries
emp_ed_5_8 = emp_ed_5_8[emp_ed_5_8['Country'].isin(target_countries)].copy()
emp_ed_5_8

,Country,2015,2018,2022
3,Belgium,45.8,49.5,52.1
4,Bulgaria,33.2,32.6,33.5
5,Czechia,23.3,24.9,27
6,Denmark,37.1,39,42.1
7,Germany,30.5,31.6,33.2
8,Estonia,38.5,39.8,42.8
9,Ireland,48.9,49.9,55.8
10,Greece,34.6,37.5,39.6
11,Spain,42.1,43.2,46.4
12,France,39.3,42.1,46.7


In [107]:
# Add the years to a variable Year and put country as index
emp_ed_5_8_long = emp_ed_5_8.melt(
    id_vars=["Country"],
    value_vars=['2015', '2018', '2022'],
    var_name="Year",
    value_name="emp_ed_5_8"
)

# define index to country_Year
emp_ed_5_8_long["index_id"] = emp_ed_5_8_long["Country"].str.replace(" ", "_") + "_" + emp_ed_5_8_long["Year"].astype(str)

# put index
emp_ed_5_8_long = emp_ed_5_8_long.set_index("index_id")

# drop redundant columns
emp_ed_5_8_long = emp_ed_5_8_long.drop(columns=['Country', 'Year'])

# change : to Nan
emp_ed_5_8_long = emp_ed_5_8_long.replace(':', np.nan)


emp_ed_5_8_long

,emp_ed_5_8
index_id,
Belgium_2015,45.8
Bulgaria_2015,33.2
Czechia_2015,23.3
Denmark_2015,37.1
Germany_2015,30.5
...,...
Romania_2022,24.3
Slovenia_2022,43.5
Slovakia_2022,31.8


# All together

In [111]:
# mudar os index_id para colunas para não dar erro dps
for df in [grad_ternary_1000_long, stu_ternary_100_long, adults_edu_train_long,
           emp_edu_train_100_long, income_0_2_long, income_3_4_long,income_5_8_long,
           gov_exp_long, prod_long, foreign_ed_0_2_long, foreign_ed_3_4_long,
           foreign_ed_5_8_long, NEET_long, emp_ed_0_2_long, emp_ed_3_4_long,
           emp_ed_5_8_long]:

    if df.index.name == "index_id":
        df.reset_index(inplace=True)

In [112]:
#dar merge em tudo

master = pisa_long.merge(
    grad_ternary_1000_long, on="index_id", how="outer"
).merge(
    stu_ternary_100_long, on="index_id", how="outer"
).set_index("index_id").merge(
    adults_edu_train_long, on="index_id", how="outer"
).merge(
    emp_edu_train_100_long, on="index_id", how="outer"
).merge(
    income_0_2_long, on="index_id", how="outer"
).merge(
    income_3_4_long, on="index_id", how="outer"
).merge(
    income_5_8_long, on="index_id", how="outer"
).merge(
    gov_exp_long, on="index_id", how="outer"
).merge(
    prod_long, on="index_id", how="outer"
).merge(
    foreign_ed_0_2_long, on="index_id", how="outer"
).merge(
    foreign_ed_3_4_long, on="index_id", how="outer"
).merge(
    foreign_ed_5_8_long, on="index_id", how="outer"
).merge(
    NEET_long, on="index_id", how="outer"
).merge(
    emp_ed_0_2_long, on="index_id", how="outer"
).merge(
    emp_ed_3_4_long, on="index_id", how="outer"
).merge(
    emp_ed_5_8_long, on="index_id", how="outer"
)

master = master.set_index("index_id")

master

,PISA_score,grad_ternary_1000,stu_ternary_100,a_edu_train_per_100,e_edu_train_per_100,income_0_2,income_3_4,income_5_8,gov_exp,prod,foreign_ed_0_2,foreign_ed_3_4,foreign_ed_5_8,NEET,emp_ed_0_2,emp_ed_3_4,emp_ed_5_8
index_id,,,,,,,,,,,,,,,,,
Austria_2015,493.907236,74.7,28.9,19.7,16.7,19286,24298,28758,10.72222,80.06,26.5,44.1,29.4,9,10.7,54.7,34.5
Austria_2018,493.989960,73.5,29.4,20.1,16.6,21290.0,26358.0,30743.0,10.68115,82.30,26.1,43.5,30.4,8.8,10.6,53.1,36.3
Austria_2022,489.922861,78.4,31.8,20.1,17.5,21587,28829,33334,9.93358,86.61,25.1,40.9,34.0,9.1,10.2,51,38.8
Belgium_2015,505.839337,79.6,36.6,13.8,6.9,16767,21920,28653,12.00727,85.44,37.0,32.8,30.2,13.6,15.3,38.3,45.8
Belgium_2018,502.546208,82.3,39.2,15.9,8.6,17672.0,24132.0,30644.0,12.18058,84.81,34.1,32.2,33.7,11.4,12.7,37.5,49.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Spain_2018,488.210239,96.3,40.3,16.2,11.2,11601.0,14882.0,20893.0,9.99577,62.15,41.2,33.1,25.7,15.3,32.8,23.9,43.2
Spain_2022,484.625645,108.0,40.5,21.2,17.1,13556,16320,22300,9.87539,63.89,42.5,31.3,26.2,12.7,29.8,23.8,46.4
Sweden_2015,495.592525,59.0,26.5,34.2,28.9,22787,27353,30508,15.18671,79.88,32.1,30.5,37.4,7.1,11.1,46.7,41.5


In [113]:
master.to_excel('master_dataset_1.4.xlsx')